In [12]:
# Import the PyTorch library
import torch
import torch.nn as nn
from torch.nn import functional as F

In [24]:
# Set the device to 'mps' if CUDA is available, otherwise set it to 'cpu' for using apple silicon
device = "mps" if torch.backends.mps.is_available() else "cpu"
# Print the device that will be used for computations
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

mps


In [14]:
# Open the file 'wizard_of_oz.txt' in read mode with utf-8 encoding
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    # Read the entire content of the file into the variable 'text'
    text = f.read()

# Create a sorted set of unique characters present in the 'text'
chars = sorted(set(text))

# Calculate the size of the vocabulary, which is the number of unique characters in the 'text'
vocab_size = len(chars)

In [15]:
# Create a dictionary to map each character to a unique integer
string_to_int = { ch:i for i, ch in enumerate(chars) }

# Create a dictionary to map each integer back to its corresponding character
int_to_string = { i:ch for i, ch in enumerate(chars) }

# Define a function to encode a string into a list of integers
encode = lambda s: [string_to_int[c] for c in s]

# Define a function to decode a list of integers back into a string
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Encode the entire text into a tensor of integers
data = torch.tensor(encode(text), dtype=torch.long)

# Print the first 100 elements of the encoded data
data[:100]

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])

In [16]:
# Calculate the number of elements that correspond to 80% of the total data length
n = int(0.8*len(data))

# Split the data into training and validation sets
train_data, val_data = data[:n], data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[65, 78,  1, 59, 68, 71, 66, 58],
        [71, 62, 67, 60,  1, 54, 73,  1],
        [78,  1, 55, 58, 54, 74, 73, 62],
        [73, 68,  1, 72, 68, 66, 58,  1]], device='mps:0')
targets:
tensor([[78,  1, 59, 68, 71, 66, 58, 57],
        [62, 67, 60,  1, 54, 73,  1, 73],
        [ 1, 55, 58, 54, 74, 73, 62, 59],
        [68,  1, 72, 68, 66, 58,  1, 69]], device='mps:0')


In [17]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
# # Set the block size to 8. This is the number of elements to consider at a time from the training data.
# block_size = 8

# # Split the training data into two parts: 'x' and 'y'. 'x' is the first 'block_size' elements and 'y' is the next 'block_size' elements.
# x, y = train_data[:block_size],  train_data[1:block_size+1]

# # Loop over the range of block_size
# for t in range(block_size):
#     # The context is the first 't+1' elements of 'x'
#     context = x[:t+1]
#     # The target is the 't'-th element of 'y'
#     target = y[t]
#     # Print the context and target
#     print('when input is', context, 'target is', target)

In [19]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index

In [20]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


]YXs]OcE*9un'5
7QHoeNy:C1W(-o 2e.-wr4e0Z72GMVdr&)!k..XqE-w.!CCdP(d*R!8jrG5hE] .2Dvj5f.,2'w,Z8_d*zccEZ3 axO*2nMY"ql0Z6O)4:7)GDQ?qDa* FNXqU8vISUL:1gWmTH*ljOOcPn;8 8.*VhF91bLR1u)G5?OHtyw.HTuNyrywA5m6JHGCHT3Kk.,j":Cs')wXO8
jDaxuY("t&hb1(nHj'z1HF&h0RnXUGL6Ql,9uDsdFCsy[,(r5!S)1DdfSCq&xmYHt*5)p;nFSCd[MgeGf4Xo,bMkHEL9'x T]EVT2T[hl;U8jv8d1o3AtSCoMLCsZrprUDaMkSmkiLxLH* KnAqPVhTE7Qob!b?6yD
79u)* 79o)YKCI
mMBI8GY*!N73;p.Z8BeAvs:l&9rhg97.w2P*SE;2Ye097*T)"b796qfSi*1()".q*tLc'xW: 44eVni_r.qd28GrpkAmf7W])iI(FaD


In [25]:
%%time
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 2.647, val loss: 2.661
step: 250, train loss: 2.640, val loss: 2.655
step: 500, train loss: 2.629, val loss: 2.639
step: 750, train loss: 2.632, val loss: 2.658
2.4078798294067383
CPU times: user 1.93 s, sys: 489 ms, total: 2.42 s
Wall time: 3.82 s


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)